In [1]:
import pandas as pd
import random
import numpy as np
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import pyodbc 

In [106]:
ds= pd.read_csv("D:\\College FCDS\\Assignments\\y2t1\\DSmethodology\\movies_data.csv")

In [107]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722796 entries, 0 to 722795
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    722796 non-null  int64  
 1   Title                 722790 non-null  object 
 2   genres                511934 non-null  object 
 3   original_language     722796 non-null  object 
 4   popularity            722796 non-null  float64
 5   production_companies  337170 non-null  object 
 6   release_date          670329 non-null  object 
 7   budget                722796 non-null  int64  
 8   revenue               722796 non-null  int64  
 9   runtime               688346 non-null  float64
 10  status                722796 non-null  object 
 11  tagline               108098 non-null  object 
 12  votes                 722796 non-null  float64
 13  vote_count            722796 non-null  int64  
 14  Actors                497689 non-null  object 
dtype

In [108]:
ds.isna().sum()

id                           0
Title                        6
genres                  210862
original_language            0
popularity                   0
production_companies    385626
release_date             52467
budget                       0
revenue                      0
runtime                  34450
status                       0
tagline                 614698
votes                        0
vote_count                   0
Actors                  225107
dtype: int64

In [109]:
# Define the list of columns you want to keep
columns_to_filter = [
    "id", "Title", "genres", "original_language", "popularity", 
    "production_companies", "release_date", "budget", "revenue", 
    "runtime", "status", "votes", "vote_count", "Actors"
]

# Select only the columns from the list
ds = ds[columns_to_filter]

# Display the filtered DataFrame
ds.describe()

,id,popularity,budget,revenue,runtime,votes,vote_count
count,722796.000000,722796.000000,7.227960e+05,7.227960e+05,6.883460e+05,722796.000000,722796.000000
mean,526917.587990,1.807185,3.924801e+05,9.776385e+05,1.989186e+02,2.443118,27.610661
std,253164.471522,18.839397,8.136181e+06,2.010908e+07,8.581987e+04,3.175498,381.523676
min,2.000000,0.600000,0.000000e+00,-1.200000e+01,0.000000e+00,0.000000,0.000000
25%,328947.750000,0.600000,0.000000e+00,0.000000e+00,5.000000e+00,0.000000,0.000000
50%,533364.500000,0.600000,0.000000e+00,0.000000e+00,5.000000e+01,0.000000,0.000000
75%,737817.250000,1.290000,0.000000e+00,0.000000e+00,9.000000e+01,5.600000,2.000000
max,968161.000000,6682.100000,5.000000e+09,2.920357e+09,5.050505e+07,10.000000,33262.000000


Correcting datatypes

In [110]:
ds=ds.dropna(subset="Title") #because its our prime factor for grouping
ds["release_date"] = pd.to_datetime(ds["release_date"], errors='coerce')
ds["original_language"]=ds["original_language"].astype("string")
ds["status"]=ds["status"].astype("string")


In [111]:
# Step 1: Create a new column with the title in lowercase and stripped of whitespace
ds['cleaned_title'] = ds['Title'].str.lower().str.strip()

# Step 2: Find duplicates in the 'cleaned_title' column
duplicates = ds[ds.duplicated(subset=['cleaned_title'], keep=False)]

# Step 3: Get the count of duplicates
duplicate_count = duplicates.shape[0]

# Display the result
print(f"Number of duplicates: {duplicate_count}")


Number of duplicates: 233131


In [112]:
# Step 1: Check for duplicates in the 'movie id' column
duplicates_id = ds[ds.duplicated(subset=['id'], keep=False)]

# Step 2: Get the count of duplicates in the 'id' column
duplicate_count_id = duplicates_id.shape[0]

# Display the result
print(f"Number of duplicate movie IDs: {duplicate_count_id}")


Number of duplicate movie IDs: 115987


In [128]:
grouped = ds.groupby('cleaned_title').agg(
    # Numeric columns: calculate the mean for columns like revenue, budget, etc.
    Title=('Title', 'first'),
    id=('id','first'),
    revenue=('revenue', 'mean'),
    budget=('budget', 'mean'),
    votes=('votes', 'mean'),
    # Other columns: get the first value for date and original title
    release_date=('release_date', 'first'),
    Actors=('Actors',"first"),
    production_companies=('production_companies','first'),
    genres=('genres','first'),
    status=('status','first'),
    original_language=('original_language','first'),
    runtime=('runtime','mean'),
    vote_count=('vote_count','mean'),
    popularity=('popularity','mean')
).reset_index()

# Display the grouped DataFrame
grouped.shape

(571382, 15)

In [129]:

# Step 2: Find duplicates in the 'cleaned_title' column
duplicates = grouped[grouped.duplicated(subset=['cleaned_title'], keep=False)]

# Step 3: Get the count of duplicates
duplicate_count = duplicates.shape[0]

# Display the result
print(f"Number of duplicates: {duplicate_count}")


Number of duplicates: 0


In [130]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571382 entries, 0 to 571381
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   cleaned_title         571382 non-null  object        
 1   Title                 571382 non-null  object        
 2   id                    571382 non-null  int64         
 3   revenue               571382 non-null  float64       
 4   budget                571382 non-null  float64       
 5   votes                 571382 non-null  float64       
 6   release_date          527231 non-null  datetime64[ns]
 7   Actors                388358 non-null  object        
 8   production_companies  269234 non-null  object        
 9   genres                403076 non-null  object        
 10  status                571382 non-null  string        
 11  original_language     571382 non-null  string        
 12  runtime               542574 non-null  float64       
 13 

In [131]:
grouped=grouped.drop(columns=["cleaned_title"])
grouped.isna().sum()

Title                        0
id                           0
revenue                      0
budget                       0
votes                        0
release_date             44151
Actors                  183024
production_companies    302148
genres                  168306
status                       0
original_language            0
runtime                  28808
vote_count                   0
popularity                   0
dtype: int64

In [132]:
average_date = grouped['release_date'].mean()
grouped['release_date'].fillna(average_date, inplace=True)

C:\Users\Mega Store\AppData\Local\Temp\ipykernel_14376\1359037826.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  grouped['release_date'].fillna(average_date, inplace=True)


In [133]:
#recency column
max_release_date = grouped["release_date"].max()
grouped["recency"] = (max_release_date - grouped["release_date"]).dt.days

In [134]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 571382 entries, 0 to 571381
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Title                 571382 non-null  object        
 1   id                    571382 non-null  int64         
 2   revenue               571382 non-null  float64       
 3   budget                571382 non-null  float64       
 4   votes                 571382 non-null  float64       
 5   release_date          571382 non-null  datetime64[ns]
 6   Actors                388358 non-null  object        
 7   production_companies  269234 non-null  object        
 8   genres                403076 non-null  object        
 9   status                571382 non-null  string        
 10  original_language     571382 non-null  string        
 11  runtime               542574 non-null  float64       
 12  vote_count            571382 non-null  float64       
 13 

In [135]:
grouped.dropna(subset=["Actors", "production_companies", "genres"], inplace=True)


In [136]:
grouped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 220498 entries, 6 to 571375
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Title                 220498 non-null  object        
 1   id                    220498 non-null  int64         
 2   revenue               220498 non-null  float64       
 3   budget                220498 non-null  float64       
 4   votes                 220498 non-null  float64       
 5   release_date          220498 non-null  datetime64[ns]
 6   Actors                220498 non-null  object        
 7   production_companies  220498 non-null  object        
 8   genres                220498 non-null  object        
 9   status                220498 non-null  string        
 10  original_language     220498 non-null  string        
 11  runtime               217245 non-null  float64       
 12  vote_count            220498 non-null  float64       
 13  popu

In [ ]:
'''# Function to handle 0 values in (revenue, budget, votes, vote_count, runtime, popularity)
def replace_zero_values(df):
    # Calculate the means of the columns (ensure mean values are calculated as floats)
    mean_revenue = df['revenue'].mean()
    mean_budget = df['budget'].mean()
    mean_votes = df['votes'].mean()
    mean_vote_count = df['vote_count'].mean()
    mean_runtime = df['runtime'].mean()
    mean_popularity = df['popularity'].mean()

    # Find the maximum value of vote_count
    max_vote_count = df['vote_count'].max()
    
    # Replace 0 values in 'revenue' with mean * random value
    df['revenue'] = df['revenue'].apply(lambda x: mean_revenue * random.random() if x == 0 else x)
    
    # Replace 0 values in 'budget' with mean * random value
    df['budget'] = df['budget'].apply(lambda x: mean_budget * random.random() if x == 0 else x)
    
    # Replace 0 values in 'votes' with mean * random value, and apply modulo 10
    df['votes'] = df['votes'].apply(lambda x: (mean_votes * random.random()) % 10 if x == 0 else x)
    
    # Replace 0 values in 'vote_count' with mean * random value, but ensure it doesn't exceed the max value
    df['vote_count'] = df['vote_count'].apply(lambda x: min(mean_vote_count * random.random(), max_vote_count) if x == 0 else x)
    
    # Replace 0 values in 'runtime' with mean * random value, but ensure it doesn't exceed 180
    df['runtime'] = df['runtime'].apply(lambda x: min(mean_runtime * random.random(), 180) if x == 0 else x)
    
    # Replace 0 values in 'popularity' with mean * random value, but ensure it doesn't exceed 100
    df['popularity'] = df['popularity'].apply(
        lambda x: min((mean_popularity * random.random()) + random.random() * 10, 100) if x == 0 else x
    )
    
    # Ensure popularity values are in the percentage range and handle low values
    df['popularity'] = df['popularity'].apply(
        lambda x: (x % 100) if x <= 100 else 100
    )

    return df

# Apply the function to the grouped DataFrame
grouped = replace_zero_values(grouped)
'''


In [137]:
# List of columns to check for 0 values
columns_to_check = ['revenue', 'budget', 'votes', 'vote_count', 'runtime', 'popularity']

# Filter out rows where any of these columns have 0 values
ds_filtered = grouped[~(grouped[columns_to_check] == 0).any(axis=1)]

In [139]:


ds_filtered.describe()

,id,revenue,budget,votes,release_date,runtime,vote_count,popularity,recency
count,9282.000000,9.282000e+03,9.282000e+03,9282.000000,9282,9274.000000,9282.000000,9282.000000,9282.000000
mean,166272.979315,5.304714e+07,1.834607e+07,5.510470,2002-07-03 11:09:29.773772160,93.966119,1134.329188,20.387307,16983.534906
min,5.000000,1.111111e-01,3.703704e-02,0.333333,1902-04-17 00:00:00,1.000000,0.111111,0.600000,9319.000000
25%,10608.000000,1.017432e+06,1.100000e+06,4.150000,1995-10-30 18:00:00,75.146617,43.000000,3.427115,12386.500000
50%,36947.000000,8.038284e+06,5.714286e+06,6.000000,2007-07-02 00:00:00,96.000000,231.714286,8.416500,15159.000000
75%,290446.500000,3.912710e+07,2.000000e+07,6.800000,2015-02-02 12:00:00,113.000000,988.000000,16.362875,19421.250000
max,967392.000000,2.799439e+09,4.600000e+08,10.000000,2023-06-28 00:00:00,842.000000,30708.000000,6682.100000,53586.000000
std,229027.779130,1.398573e+08,3.428433e+07,1.861016,NaN,33.812338,2547.798136,141.811502,6573.102923


In [2]:
#ds_filtered.to_csv("D:\\College FCDS\\Assignments\\y2t1\\DSmethodology\\AccurateVersion.csv", index=False)
dataset= pd.read_csv("D:\\College FCDS\\Assignments\\y2t1\\DSmethodology\\AccurateVersion.csv")

dataset["Net_profit"]=dataset["revenue"]-dataset["budget"]

In [10]:
conn= pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                          'SERVER=localhost;'
                          'DATABASE=MovieDB;'
                          'UID=amr;'
                          'PWD=dbassign1'
                          )
cursor = conn.cursor()

OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [2].  (2) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (2)')